In [1]:
#Copy data to collab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import shutil
import pandas as pd

In [3]:
destination_path = '/content/emailsubjects'
source_path = '/content/drive/My Drive/emailsubjects/'

# Copy the file
shutil.copytree(source_path, destination_path,dirs_exist_ok=True)

'/content/emailsubjects'

In [4]:
#test_df = pd.read_excel('emailsubjects/aiml-qa-test.xlsx')

In [5]:
test_df = pd.read_excel('emailsubjects/aiml-qa-test.xlsx')
dev_df = pd.read_excel('emailsubjects/aiml-qa-dev.xlsx')
train_df = pd.read_excel('emailsubjects/aiml-qa-train.xlsx')

In [6]:
train_df = pd.concat([train_df,dev_df])

In [7]:
train_df['output'] = train_df[['answer', 'answer1', 'answer2']].bfill(axis=1).iloc[:, 0]

In [8]:
test_df.head()

,question,answer1,answer2
0,How we can effectively convert 2D images to 1D?,Converting images to 1D data may not be effect...,"To effectively convert 2D images to 1D, use te..."
1,Can we utilize an autoencoder to perform dimen...,"Yes, autoencoders can be applied to numerical ...","Yes, autoencoders can be used for dimensionali..."
2,What is NLP's current biggest challenge that i...,The main challenges of NLP is finding and coll...,NLP models struggle with tasks that require re...
3,Which problems cannot be solved by Neural netw...,While neural networks have shown great success...,"Neural networks are powerful, but they may str..."
4,Is scaling necessary for SVM?,"Yes, scaling the input data is generally recom...",Scaling the input data is advisable when utili...


In [9]:
test_df['output'] = test_df[['answer1', 'answer2']].bfill(axis=1).iloc[:, 0]

In [10]:
train_df.rename(columns={'question': 'input'}, inplace=True)
test_df.rename(columns={'question': 'input'}, inplace=True)

In [11]:
train_df.drop(['answer', 'answer1', 'answer2'], axis=1, inplace=True)
test_df.drop(['answer1', 'answer2'], axis=1, inplace=True)

In [12]:
#!pip install pyarrow==15.0.2

In [13]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [14]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [15]:
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth



In [16]:
model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    # model_name = "unsloth/Qwen2.5-7B",
    model_name = "unsloth/Qwen2.5-0.5B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.9.post2: Fast Qwen2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/457M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.87k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [17]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post2 patched 24 layers with 0 QKV layers, 24 O layers and 24 MLP layers.


In [18]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [19]:
output_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Input:
{}

### Response:
{}"""


In [20]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    #instructions = examples["instruction"]
    instructions = examples["input"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass



In [21]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_output(examples):
    #instructions = examples["instruction"]
    instructions = examples["input"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = output_prompt.format(input, output)
        texts.append(text)
    return { "text" : texts, }
pass


In [22]:
from datasets import Dataset

In [28]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=d4b39d56509ca877986e32df5023496d89c833c5d864a427ccb02b5f0be89b8f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [26]:
!pip install --upgrade datasets

In [29]:
train_dataset = Dataset.from_pandas(train_df)

In [30]:
from datasets import load_dataset
#dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = train_dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1396 [00:00<?, ? examples/s]

In [31]:
test_df['given_output'] = test_df['output']
test_df['output'] = ''

In [32]:
test_dataset = Dataset.from_pandas(test_df)

In [33]:
test_ds = test_dataset.map(formatting_prompts_output, batched = True,)

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

In [34]:
test_df

,input,output,given_output
0,How we can effectively convert 2D images to 1D?,,Converting images to 1D data may not be effect...
1,Can we utilize an autoencoder to perform dimen...,,"Yes, autoencoders can be applied to numerical ..."
2,What is NLP's current biggest challenge that i...,,The main challenges of NLP is finding and coll...
3,Which problems cannot be solved by Neural netw...,,While neural networks have shown great success...
4,Is scaling necessary for SVM?,,"Yes, scaling the input data is generally recom..."
...,...,...,...
115,Can you repeat difference between data mining ...,,Data mining refers to the process of discoveri...
116,Is there any software available for clinical l...,,"CLAMP (Clinical Language Annotation, Modeling,..."
117,When do we slice?,,Slicing is a useful technique in Python for ex...
118,"In terms of obtaining better context, is lemma...",,"Yes, lemmatization is generally considered bet..."


In [35]:
test_ds

Dataset({
    features: ['input', 'output', 'given_output', 'text'],
    num_rows: 120
})

In [36]:
dataset[0]

{'input': 'What is the difference between concatenation vs. summation of two tensors?',
 'output': 'Concatenation combines two tensors by adding them together along a specified dimension. Summation adds the elements of two tensors together element-wise.',
 '__index_level_0__': 0,
 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is the difference between concatenation vs. summation of two tensors?\n\n### Input:\nWhat is the difference between concatenation vs. summation of two tensors?\n\n### Response:\nConcatenation combines two tensors by adding them together along a specified dimension. Summation adds the elements of two tensors together element-wise.<|endoftext|>'}

In [37]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported



In [38]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1396 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [39]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,396 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 8,798,208


Step,Training Loss
1,2.438600
2,2.492400
3,2.416400
4,2.411700
5,2.086100
6,1.466500
7,1.509800
8,1.337200
9,1.400800
10,1.263900


In [40]:
test_df.iloc[0]['input']

'How we can effectively convert 2D images to 1D?'

In [41]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.", # instruction
        "How we can effectively convert 2D images to 1D?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Input:\nHow we can effectively convert 2D images to 1D?\n\n### Response:\nWe can use convolutional neural networks (CNNs) to extract features from 2D images and then convert them to 1D representations.<|endoftext|>']

In [42]:
train_df.iloc[0]['input']

'What is the difference between concatenation vs. summation of two tensors?'

In [43]:
train_df.iloc[0]['output']

'Concatenation combines two tensors by adding them together along a specified dimension. Summation adds the elements of two tensors together element-wise.'

In [44]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.", # instruction
        "What is the difference between concatenation vs. summation of two tensors?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Input:\nWhat is the difference between concatenation vs. summation of two tensors?\n\n### Response:\nConcatenation involves combining two tensors along a new dimension, while summation involves adding corresponding elements from two tensors.<|endoftext|>']

In [45]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [46]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [48]:
model.push_to_hub("sbtraining2020/qwen_qa1", token = hf_token) # Online saving
tokenizer.push_to_hub("sbtraining2020/qwen_qa1", token = hf_token) # Online saving


README.md:   0%|          | 0.00/585 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/35.2M [00:00<?, ?B/s]

Saved model to https://huggingface.co/sbtraining2020/qwen_qa1


In [49]:
test_ds

Dataset({
    features: ['input', 'output', 'given_output', 'text'],
    num_rows: 120
})

In [50]:
def get_answer(answer):
  # Extract the content after "### Response:"
  response_start = answer.find("### Response:") + len("### Response:")
  response_end = answer.find("<|endoftext|>")
  response = answer[response_start:response_end].strip()
  return response

In [51]:
test_ds[0]['text']

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Input:\nHow we can effectively convert 2D images to 1D?\n\n### Response:\n'

In [52]:
from transformers import TextStreamer

In [53]:
original_answers = []
qwen_answers = []

for index, row in enumerate(test_ds):
    original_answer = row['given_output']
    Prompt = row['text']

    # Define the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move both models to the same device
    model.to(device)
    inputs = tokenizer(
    [
        Prompt
    ], return_tensors = "pt").to("cuda")

    text_streamer = TextStreamer(tokenizer)
    generated_ids = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)
    # Get the generated text by decoding the IDs
    qwen_answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True) # Apply to the decoded text
    qwen_answers.append(get_answer(qwen_answer))


    original_answers.append(original_answer)

zipped_subjects = list(zip(original_answers, qwen_answers))
qwen_result_df = pd.DataFrame(zipped_subjects, columns = ['original_answers', 'qwen_answers'])


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Input:
How we can effectively convert 2D images to 1D?

### Response:
We can use convolutional neural networks (CNNs) to extract features from 2D images and then convert them to 1D representations.<|endoftext|>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Input:
Can we utilize an autoencoder to perform dimensionality reduction on numerical datasets?

### Response:


In [54]:
test_df

,input,output,given_output
0,How we can effectively convert 2D images to 1D?,,Converting images to 1D data may not be effect...
1,Can we utilize an autoencoder to perform dimen...,,"Yes, autoencoders can be applied to numerical ..."
2,What is NLP's current biggest challenge that i...,,The main challenges of NLP is finding and coll...
3,Which problems cannot be solved by Neural netw...,,While neural networks have shown great success...
4,Is scaling necessary for SVM?,,"Yes, scaling the input data is generally recom..."
...,...,...,...
115,Can you repeat difference between data mining ...,,Data mining refers to the process of discoveri...
116,Is there any software available for clinical l...,,"CLAMP (Clinical Language Annotation, Modeling,..."
117,When do we slice?,,Slicing is a useful technique in Python for ex...
118,"In terms of obtaining better context, is lemma...",,"Yes, lemmatization is generally considered bet..."


In [55]:
qwen_result_df

,original_answers,qwen_answers
0,Converting images to 1D data may not be effect...,We can use convolutional neural networks (CNNs...
1,"Yes, autoencoders can be applied to numerical ...","Yes, an autoencoder can be used to perform dim..."
2,The main challenges of NLP is finding and coll...,The current biggest challenge in Natural Langu...
3,While neural networks have shown great success...,Neural networks are not capable of solving all...
4,"Yes, scaling the input data is generally recom...","Scaling is not necessary for SVM, as it is a n..."
...,...,...
115,Data mining refers to the process of discoveri...,Data mining and machine learning are two disti...
116,"CLAMP (Clinical Language Annotation, Modeling,...","Yes, there are several software available for ..."
117,Slicing is a useful technique in Python for ex...,"When we slice, we select a specific portion of..."
118,"Yes, lemmatization is generally considered bet...",Lemmatization is generally considered superior...


In [121]:
qwen_result_df['original_answers'] = test_df['given_output']

In [122]:
qwen_result_df

,original_answers,qwen_answers
0,Converting images to 1D data may not be effect...,We can use convolutional neural networks (CNNs...
1,"Yes, autoencoders can be applied to numerical ...","Yes, an autoencoder can be used to perform dim..."
2,The main challenges of NLP is finding and coll...,The current biggest challenge in Natural Langu...
3,While neural networks have shown great success...,Neural networks are not capable of solving all...
4,"Yes, scaling the input data is generally recom...","Scaling is not necessary for SVM, as it is a n..."
...,...,...
115,Data mining refers to the process of discoveri...,Data mining and machine learning are two disti...
116,"CLAMP (Clinical Language Annotation, Modeling,...","Yes, there are several software available for ..."
117,Slicing is a useful technique in Python for ex...,"When we slice, we select a specific portion of..."
118,"Yes, lemmatization is generally considered bet...",Lemmatization is generally considered superior...


In [56]:
test_subjects = qwen_result_df['original_answers']
model_output = qwen_result_df['qwen_answers']

In [57]:
#!pip install datasets

In [58]:
#!pip install rouge_score

In [59]:
#!pip install --upgrade datasets

In [60]:
#!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [63]:
from rouge_score import rouge_scorer

def calculate_rouge(actual, predicted):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    return scorer.score(actual, predicted)

In [64]:
# Apply ROUGE score calculation row by row
def compute_rouge_for_row(row):
    return calculate_rouge(row['original_answers'], row['qwen_answers'])

qwen_result_df['rouge_scores'] = qwen_result_df.apply(compute_rouge_for_row, axis=1)

In [65]:
qwen_result_df

,original_answers,qwen_answers,rouge_scores
0,Converting images to 1D data may not be effect...,We can use convolutional neural networks (CNNs...,"{'rouge1': (0.4, 0.2962962962962963, 0.3404255..."
1,"Yes, autoencoders can be applied to numerical ...","Yes, an autoencoder can be used to perform dim...","{'rouge1': (0.6923076923076923, 0.310344827586..."
2,The main challenges of NLP is finding and coll...,The current biggest challenge in Natural Langu...,"{'rouge1': (0.2, 0.20588235294117646, 0.202898..."
3,While neural networks have shown great success...,Neural networks are not capable of solving all...,"{'rouge1': (0.2, 0.14285714285714285, 0.166666..."
4,"Yes, scaling the input data is generally recom...","Scaling is not necessary for SVM, as it is a n...","{'rouge1': (0.2222222222222222, 0.121212121212..."
...,...,...,...
115,Data mining refers to the process of discoveri...,Data mining and machine learning are two disti...,"{'rouge1': (0.21621621621621623, 0.53333333333..."
116,"CLAMP (Clinical Language Annotation, Modeling,...","Yes, there are several software available for ...","{'rouge1': (0.3125, 0.18518518518518517, 0.232..."
117,Slicing is a useful technique in Python for ex...,"When we slice, we select a specific portion of...","{'rouge1': (0.3333333333333333, 0.210526315789..."
118,"Yes, lemmatization is generally considered bet...",Lemmatization is generally considered superior...,"{'rouge1': (0.35714285714285715, 0.71428571428..."


In [67]:
# Initialize variables to accumulate scores
total_rouge1_precision = 0
total_rouge1_recall = 0
total_rouge1_f1 = 0

total_rouge2_precision = 0
total_rouge2_recall = 0
total_rouge2_f1 = 0

total_rougeL_precision = 0
total_rougeL_recall = 0
total_rougeL_f1 = 0

n = len(qwen_result_df)  # number of rows in the dataset

# Loop through the DataFrame to accumulate the ROUGE scores
for scores in qwen_result_df['rouge_scores']:
    total_rouge1_precision += scores['rouge1'].precision
    total_rouge1_recall += scores['rouge1'].recall
    total_rouge1_f1 += scores['rouge1'].fmeasure

    total_rouge2_precision += scores['rouge2'].precision
    total_rouge2_recall += scores['rouge2'].recall
    total_rouge2_f1 += scores['rouge2'].fmeasure

    total_rougeL_precision += scores['rougeL'].precision
    total_rougeL_recall += scores['rougeL'].recall
    total_rougeL_f1 += scores['rougeL'].fmeasure

# Calculate average ROUGE scores across the dataset
average_rouge1 = {
    'precision': total_rouge1_precision / n,
    'recall': total_rouge1_recall / n,
    'f1': total_rouge1_f1 / n
}

average_rouge2 = {
    'precision': total_rouge2_precision / n,
    'recall': total_rouge2_recall / n,
    'f1': total_rouge2_f1 / n
}

average_rougeL = {
    'precision': total_rougeL_precision / n,
    'recall': total_rougeL_recall / n,
    'f1': total_rougeL_f1 / n
}

# Print the final averaged ROUGE scores
print("Average ROUGE-1:", average_rouge1)
print("Average ROUGE-2:", average_rouge2)
print("Average ROUGE-L:", average_rougeL)

Average ROUGE-1: {'precision': 0.4409976422402672, 'recall': 0.3784543991185402, 'f1': 0.38432054382424047}
Average ROUGE-2: {'precision': 0.2079923741865798, 'recall': 0.1719421010222892, 'f1': 0.17660562447762124}
Average ROUGE-L: {'precision': 0.3669665944856232, 'recall': 0.3126798725722046, 'f1': 0.3183124591201149}
